In [1]:
## imports
import pandas as pd

In [5]:
## loading and reading in csv dataset using pandas
brazil_df = pd.read_csv('brazil_covid19.csv')
brazil_df.head()

,date,region,state,cases,deaths
0,2020-02-26,Sudeste,São Paulo,1,0
1,2020-02-27,Sudeste,São Paulo,1,0
2,2020-02-28,Sudeste,São Paulo,1,0
3,2020-02-29,Sudeste,São Paulo,2,0
4,2020-03-01,Sudeste,São Paulo,2,0
